In [194]:
install.packages("MASS")
library(MASS)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [195]:
#import data of px with prior stroke
strokedata <- read.csv("prior_stroke_model.csv")

In [196]:
#create dataframe from data
df <- data.frame(strokedata) 

In [197]:
# check number of rows - should be 1603 px with prior stroke
nrow(df)

[1] 1603

In [198]:
# create 2 separate dataframes, 1 has depressed px, other has non-depressed px
data_cevd <- df[df$dep_diag_yes == 1, ]
data_cevdno <- df[df$dep_diag_yes == 0, ]

In [199]:
# check number of rows - should be 155 with prior depression diagnosis (pre-stroke)
nrow(data_cevd)

[1] 155

In [200]:
# check number of rows - should be 1448 with prior depression diagnosis (pre-stroke)
nrow(data_cevdno)

[1] 1448

In [201]:
# make variables into factors or filtered variables
data_cevd$p20510 <- factor(data_cevd$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
data_cevdno$p20510 <- factor(data_cevdno$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
data_cevd$sleep_filter <- ifelse(data_cevd$pa_sleep_convert_hr >= 7 & data_cevd$pa_sleep_convert_hr <= 9, 1, 0)
data_cevdno$sleep_filter <- ifelse(data_cevdno$pa_sleep_convert_hr >= 7 & data_cevdno$pa_sleep_convert_hr <=9, 1, 0)
#check it was successful
head(data_cevdno$sleep_filter, 30)

[1] 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1

In [207]:
model_dy <- polr(p20510 ~  card_sum_filter +hchol_yes +  diab_yes+ study_t2e+ white_yes+ acc_age + male +sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = data_cevd, Hess = TRUE)
model_dn <- polr(p20510 ~  card_sum_filter+hchol_yes + diab_yes+ study_t2e+  white_yes+ acc_age + male+ sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = data_cevdno, Hess = TRUE)

Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”
Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”


In [208]:
df_model_dy <- summary(model_dy)
#get odds ratios
odds_ratios_dy <- exp(df_model_dy$coefficients[, "Value"])
#get standard errors
stde_dy <- df_model_dy$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dy <- exp(cbind(
  OR = df_model_dy$coefficients[, "Value"],
  lower = df_model_dy$coefficients[, "Value"] - 1.96 * stde_dy,
  upper = df_model_dy$coefficients[, "Value"] + 1.96 * stde_dy 
))

print(conf_int_dy)

                          OR      lower      upper
hchol_yes         0.50358105 0.24222535 1.04693370
diab_yes          2.62201318 0.91872793 7.48312195
study_t2e         0.77368386 0.47791622 1.25249300
white_yes         0.37328773 0.04876908 2.85721485
acc_age           0.96663995 0.92505738 1.01009172
male              0.76692306 0.37583588 1.56496764
sleep_filter      0.48585478 0.22930242 1.02944774
pa_sed_convert    1.00300796 0.99946729 1.00656117
pa_light_convert  0.99758721 0.99388003 1.00130822
pa_mv_convert_log 0.86058345 0.63469181 1.16687165
1|2               0.03862229 0.02339164 0.06376983
2|3               0.37348655 0.17074384 0.81696770
3|4               0.89920342 0.34363297 2.35299538


In [209]:
#make a table of coefficients
ctable_dy <- coef(summary(model_dy))
#get p values
p_dy <- pnorm(abs(ctable_dy[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dy <- cbind(ctable_dy, "p value" = p_dy))

,Value,Std. Error,t value,p value
hchol_yes,-0.686010601,0.373406226,-1.8371697,6.618485e-02
diab_yes,0.963942413,0.535054931,1.8015765,7.161205e-02
study_t2e,-0.256591932,0.245779540,-1.0439922,2.964890e-01
white_yes,-0.985405768,1.038394434,-0.9489706,3.426356e-01
acc_age,-0.033929186,0.022433841,-1.5124109,1.304294e-01
male,-0.265368791,0.363894866,-0.7292458,4.658513e-01
sleep_filter,-0.721845515,0.383095919,-1.8842422,5.953222e-02
pa_sed_convert,0.003003442,0.001804231,1.6646656,9.597950e-02
pa_light_convert,-0.002415701,0.001899525,-1.2717397,2.034656e-01
pa_mv_convert_log,-0.150144687,0.155342373,-0.9665405,3.337738e-01


In [212]:
df_model_dn <- summary(model_dn)
#get odds ratios
odds_ratios_dn <- exp(df_model_dn$coefficients[, "Value"])
#get standard errors
stde_dn <- df_model_dn$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dn <- exp(cbind(
  OR = df_model_dn$coefficients[, "Value"],
  lower = df_model_dn$coefficients[, "Value"] - 1.96 * stde_dn,
  upper = df_model_dn$coefficients[, "Value"] + 1.96 * stde_dn 
))

print(conf_int_dn)


                         OR     lower     upper
hchol_yes         1.0092582 0.7611973 1.3381579
diab_yes          1.1148429 0.7123923 1.7446493
study_t2e         1.0817931 0.8839403 1.3239314
white_yes         1.1777513 0.5206143 2.6643488
acc_age           0.9655556 0.9498226 0.9815493
male              0.6481931 0.4869224 0.8628773
sleep_filter      0.8815599 0.6548284 1.1867961
pa_sed_convert    1.0003175 0.9990416 1.0015950
pa_light_convert  0.9992721 0.9977587 1.0007878
pa_mv_convert_log 0.7702306 0.6855854 0.8653264
1|2               0.1863077 0.1483181 0.2340277
2|3               1.2503077 0.8883964 1.7596530
3|4               2.7908439 1.7767466 4.3837481


In [211]:

#make a table of coefficients
ctable_dn <- coef(summary(model_dn))
#get p values
p_dn <- pnorm(abs(ctable_dn[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dn <- cbind(ctable_dn, "p value" = p_dn))

,Value,Std. Error,t value,p value
hchol_yes,0.0092156352,0.1439175205,0.06403414,9.489430e-01
diab_yes,0.1087135182,0.2284898237,0.47579151,6.342229e-01
study_t2e,0.0786199227,0.1030539299,0.76290077,4.455226e-01
white_yes,0.1636069296,0.4165064960,0.39280763,6.944616e-01
acc_age,-0.0350515615,0.0083818754,-4.18182803,2.891747e-05
male,-0.4335666694,0.1459611404,-2.97042534,2.973877e-03
sleep_filter,-0.1260623511,0.1516937155,-0.83103213,4.059555e-01
pa_sed_convert,0.0003174369,0.0006511564,0.48749720,6.259060e-01
pa_light_convert,-0.0007281600,0.0007732849,-0.94164521,3.463743e-01
pa_mv_convert_log,-0.2610653829,0.0593963567,-4.39530970,1.106149e-05


In [213]:
#full data for all patients who have no prior stroke
controldata <- read.csv("control_model.csv")

In [214]:
control_df <- data.frame(controldata) 

In [215]:
# check number of rows - should be 68685 without a prior stroke
nrow(control_df)

[1] 68685

In [216]:
# create 2 separate dataframes, 1 has depressed px, other has non-depressed px
controldata_cevd <- control_df[control_df$dep_diag_yes == 1, ]
controldata_cevdno <- control_df[control_df$dep_diag_yes == 0, ]

In [217]:
#should be 6096
nrow(controldata_cevd)

[1] 6096

In [218]:
#should be 62589
nrow(controldata_cevdno)

[1] 62589

In [219]:
#create variables from factoring and filter controls
controldata_cevd$p20510 <- factor(controldata_cevd$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
controldata_cevdno$p20510 <- factor(controldata_cevdno$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
controldata_cevd$sleep_filter <- ifelse(controldata_cevd$pa_sleep_convert_hr >= 7 & controldata_cevd$pa_sleep_convert_hr <= 9, 1, 0)
controldata_cevdno$sleep_filter <- ifelse(controldata_cevdno$pa_sleep_convert_hr >= 7 & controldata_cevdno$pa_sleep_convert_hr <=9, 1, 0)

In [220]:
model_dy <- polr(p20510 ~ card_sum_filter +hchol_yes +  diab_yes+ study_t2e+   white_yes+ acc_age + male +sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = controldata_cevd, Hess = TRUE)
model_dn <- polr(p20510 ~ card_sum_filter +hchol_yes +  diab_yes+ study_t2e+   white_yes+ acc_age + male+ sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = controldata_cevdno, Hess = TRUE)

Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”
Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”


In [222]:
df_model_dy <- summary(model_dy)
#get odds ratios
odds_ratios_dy <- exp(df_model_dy$coefficients[, "Value"])
#get standard errors
stde_dy <- df_model_dy$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dy <- exp(cbind(
  OR = df_model_dy$coefficients[, "Value"],
  lower = df_model_dy$coefficients[, "Value"] - 1.96 * stde_dy,
  upper = df_model_dy$coefficients[, "Value"] + 1.96 * stde_dy 
))
print(conf_int_dy)

                          OR      lower      upper
hchol_yes         1.16308726 1.01805640 1.32877900
diab_yes          1.43291210 1.12336488 1.82775619
study_t2e         1.08179692 1.00466426 1.16485141
white_yes         0.92575461 0.69144055 1.23947258
acc_age           0.96472300 0.95922148 0.97025607
male              1.09235980 0.97527050 1.22350664
sleep_filter      1.02218650 0.91617554 1.14046402
pa_sed_convert    1.00003835 0.99957038 1.00050653
pa_light_convert  0.99983194 0.99927221 1.00039199
pa_mv_convert_log 0.81555437 0.78008214 0.85263961
1|2               0.08292241 0.07739713 0.08884213
2|3               0.60524616 0.54564963 0.67135191
3|4               1.34597146 1.18009675 1.53516157


In [223]:
#make a table of coefficients
ctable_dy <- coef(summary(model_dy))
#get p values
p_dy <- pnorm(abs(ctable_dy[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dy <- cbind(ctable_dy, "p value" = p_dy))

,Value,Std. Error,t value,p value
hchol_yes,0.1510779007,0.0679502952,2.2233590,2.619160e-02
diab_yes,0.3597088104,0.1241736104,2.8968217,3.769639e-03
study_t2e,0.0786234754,0.0377398259,2.0833025,3.722365e-02
white_yes,-0.0771460799,0.1488938916,-0.5181279,6.043690e-01
acc_age,-0.0359142639,0.0029178639,-12.3084095,8.161570e-35
male,0.0883403129,0.0578473065,1.5271292,1.267289e-01
sleep_filter,0.0219439613,0.0558628835,0.3928183,6.944537e-01
pa_sed_convert,0.0000383451,0.0002388054,0.1605705,8.724317e-01
pa_light_convert,-0.0001680723,0.0002857060,-0.5882699,5.563512e-01
pa_mv_convert_log,-0.2038871862,0.0226881989,-8.9864862,2.552608e-19


In [224]:
df_model_dn <- summary(model_dn)
#get odds ratios
odds_ratios_dn <- exp(df_model_dn$coefficients[, "Value"])
#get standard errors
stde_dn <- df_model_dn$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dn <- exp(cbind(
  OR = df_model_dn$coefficients[, "Value"],
  lower = df_model_dn$coefficients[, "Value"] - 1.96 * stde_dn,
  upper = df_model_dn$coefficients[, "Value"] + 1.96 * stde_dn 
))
print(conf_int_dn)

                         OR     lower     upper
hchol_yes         1.0610889 0.9992826 1.1267180
diab_yes          1.2347396 1.0976645 1.3889327
study_t2e         1.0013975 0.9724317 1.0312262
white_yes         0.9479395 0.8558139 1.0499822
acc_age           0.9646010 0.9624628 0.9667439
male              0.8114990 0.7769197 0.8476174
sleep_filter      0.9472434 0.9068133 0.9894761
pa_sed_convert    1.0002906 1.0001083 1.0004729
pa_light_convert  0.9997576 0.9995406 0.9999747
pa_mv_convert_log 0.8650631 0.8491652 0.8812586
1|2               0.2651575 0.2609231 0.2694605
2|3               2.6439879 2.5121603 2.7827333
3|4               7.0758318 6.5172097 7.6823362


In [225]:

#make a table of coefficients
ctable_dn <- coef(summary(model_dn))
#get p values
p_dn <- pnorm(abs(ctable_dn[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dn <- cbind(ctable_dn, "p value" = p_dn))

,Value,Std. Error,t value,p value
hchol_yes,0.0592956512,3.061905e-02,1.93656063,5.279907e-02
diab_yes,0.2108601377,6.003849e-02,3.51208268,4.446097e-04
study_t2e,0.0013965734,1.497552e-02,0.09325707,9.256993e-01
white_yes,-0.0534645527,5.216213e-02,-1.02496865,3.053780e-01
acc_age,-0.0360407845,1.132201e-03,-31.83248748,2.300204e-222
male,-0.2088721081,2.221742e-02,-9.40127643,5.390510e-21
sleep_filter,-0.0541991530,2.225484e-02,-2.43538727,1.487586e-02
pa_sed_convert,0.0002905294,9.297034e-05,3.12496907,1.778238e-03
pa_light_convert,-0.0002424287,1.107618e-04,-2.18873850,2.861585e-02
pa_mv_convert_log,-0.1449528473,9.463640e-03,-15.31681682,5.903884e-53


In [226]:
#full data for all patients who have a future ischemic stroke
is_data <- read.csv("incidentstroke_df.csv")

In [227]:
#make a df for IS data
is_df <- data.frame(is_data) 
#check number of rows - should be 1173
nrow(is_df)

[1] 1173

In [228]:
# create 2 separate dataframes, 1 has depressed px, other has non-depressed px
isdata_cevd <- is_df[is_df$dep_diag_yes == 1, ]
isdata_cevdno <- is_df[is_df$dep_diag_yes == 0, ]

In [229]:
#check numbers - should be 100
nrow(isdata_cevd)

[1] 100

In [230]:
#check numbers - should be 1073
nrow(isdata_cevdno)

[1] 1073

In [231]:
#create variables with factoring and filtering
isdata_cevd$p20510 <- factor(isdata_cevd$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
isdata_cevdno$p20510 <- factor(isdata_cevdno$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
isdata_cevd$ethnicity <- ifelse(isdata_cevd$ethnicity > 0, 1, 0)
isdata_cevdno$ethnicity <- ifelse(isdata_cevdno$ethnicity > 0, 1, 0)
isdata_cevd$sleep_filter <- ifelse(isdata_cevd$pa_sleep_convert_hr >= 7 & isdata_cevd$pa_sleep_convert_hr <= 9, 1, 0)
isdata_cevdno$sleep_filter <- ifelse(isdata_cevdno$pa_sleep_convert_hr >= 7 & isdata_cevdno$pa_sleep_convert_hr <=9, 1, 0)

In [232]:
model_dy <- polr(p20510 ~  card_sum_filter +hchol_yes +  diab_yes+ study_t2e+ white_yes + acc_age + male +sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = isdata_cevd, Hess = TRUE)
model_dn <- polr(p20510 ~ card_sum_filter +hchol_yes +  diab_yes+ study_t2e+ white_yes + acc_age + male+ sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = isdata_cevdno, Hess = TRUE)

Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”
Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”


In [234]:
df_model_dy <- summary(model_dy)
#get odds ratios
odds_ratios_dy <- exp(df_model_dy$coefficients[, "Value"])
#get standard errors
stde_dy <- df_model_dy$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dy <- exp(cbind(
  OR = df_model_dy$coefficients[, "Value"],
  lower = df_model_dy$coefficients[, "Value"] - 1.96 * stde_dy,
  upper = df_model_dy$coefficients[, "Value"] + 1.96 * stde_dy 
))
print(conf_int_dy)

                            OR        lower        upper
hchol_yes         1.366486e+00 4.640430e-01 4.023947e+00
diab_yes          7.687180e-01 1.545459e-01 3.823636e+00
study_t2e         1.477457e+00 7.823057e-01 2.790315e+00
white_yes         8.979685e+05 3.298582e+04 2.444527e+07
acc_age           9.592472e-01 9.010016e-01 1.021258e+00
male              1.618332e+00 6.754194e-01 3.877590e+00
sleep_filter      1.619905e+00 6.435563e-01 4.077484e+00
pa_sed_convert    1.004183e+00 9.981583e-01 1.010245e+00
pa_light_convert  1.002675e+00 9.962777e-01 1.009113e+00
pa_mv_convert_log 8.681865e-01 6.029996e-01 1.249997e+00
1|2               3.657908e+06 1.343719e+05 9.957657e+07
2|3               5.205134e+07 1.684849e+06 1.608062e+09
3|4               7.692291e+07 2.401892e+06 2.463531e+09


In [235]:

#make a table of coefficients
ctable_dy <- coef(summary(model_dy))
#get p values
p_dy <- pnorm(abs(ctable_dy[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dy <- cbind(ctable_dy, "p value" = p_dy))

,Value,Std. Error,t value,p value
hchol_yes,0.312242614,0.551030950,0.5666517,5.709509e-01
diab_yes,-0.263031076,0.818486108,-0.3213629,7.479354e-01
study_t2e,0.390322419,0.324404156,1.2031980,2.288997e-01
white_yes,13.707890246,1.685743415,8.1316588,4.234554e-16
acc_age,-0.041606437,0.031960131,-1.3018231,1.929769e-01
male,0.481396276,0.445825344,1.0797867,2.802372e-01
sleep_filter,0.482367203,0.470976001,1.0241864,3.057473e-01
pa_sed_convert,0.004174516,0.003070385,1.3596066,1.739544e-01
pa_light_convert,0.002671386,0.003265633,0.8180301,4.133400e-01
pa_mv_convert_log,-0.141348772,0.185964312,-0.7600855,4.472035e-01


In [236]:
df_model_dn <- summary(model_dn)
#get odds ratios
odds_ratios_dn <- exp(df_model_dn$coefficients[, "Value"])
#get standard errors
stde_dn <- df_model_dn$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dn <- exp(cbind(
  OR = df_model_dn$coefficients[, "Value"],
  lower = df_model_dn$coefficients[, "Value"] - 1.96 * stde_dn,
  upper = df_model_dn$coefficients[, "Value"] + 1.96 * stde_dn 
))
print(conf_int_dn)

                         OR     lower      upper
hchol_yes         1.5694275 1.0574832  2.3292120
diab_yes          0.9890082 0.4751320  2.0586643
study_t2e         0.8594103 0.6769008  1.0911288
white_yes         2.3532968 0.8495922  6.5184281
acc_age           0.9632279 0.9446179  0.9822045
male              0.6092550 0.4341388  0.8550070
sleep_filter      0.7734710 0.5453013  1.0971135
pa_sed_convert    0.9995471 0.9980597  1.0010368
pa_light_convert  0.9993326 0.9974905  1.0011780
pa_mv_convert_log 0.9619771 0.8316904  1.1126735
1|2               0.2778664 0.2134102  0.3617902
2|3               2.6781473 1.6950480  4.2314277
3|4               8.5617645 4.1158377 17.8101803


In [237]:

#make a table of coefficients
ctable_dn <- coef(summary(model_dn))
#get p values
p_dn <- pnorm(abs(ctable_dn[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dn <- cbind(ctable_dn, "p value" = p_dn))

,Value,Std. Error,t value,p value
hchol_yes,0.4507109031,0.2014383220,2.23746355,2.525606e-02
diab_yes,-0.0110526555,0.3740357242,-0.02954973,9.764262e-01
study_t2e,-0.1515088754,0.1217967719,-1.24394820,2.135186e-01
white_yes,0.8558172239,0.5198041009,1.64642261,9.967678e-02
acc_age,-0.0374652800,0.0099538489,-3.76389882,1.672846e-04
male,-0.4955182980,0.1728942395,-2.86601971,4.156684e-03
sleep_filter,-0.2568670626,0.1783416916,-1.44030855,1.497801e-01
pa_sed_convert,-0.0004529671,0.0007598081,-0.59615990,5.510684e-01
pa_light_convert,-0.0006676486,0.0009413254,-0.70926436,4.781604e-01
pa_mv_convert_log,-0.0387646549,0.0742501890,-0.52208157,6.016135e-01


In [238]:
#full data for all patients who have a future CeVD
icdata <- read.csv("incident_cevd_sort_model.csv")
ic_df <- data.frame(icdata) 
# check number of rows - should be 1678
nrow(ic_df)

[1] 1678

In [239]:
# create 2 separate dataframes, 1 has depressed px, other has non-depressed px
icdata_cevd <- ic_df[ic_df$dep_diag_yes == 1, ]
icdata_cevdno <- ic_df[ic_df$dep_diag_yes == 0, ]

In [240]:
#check number - should be 152
nrow(icdata_cevd)

[1] 152

In [241]:
#check number - should be 1526
nrow(icdata_cevdno)

[1] 1526

In [242]:
# Convert the response variable to a factor with ordered levels
icdata_cevd$p20510 <- factor(icdata_cevd$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
icdata_cevdno$p20510 <- factor(icdata_cevdno$p20510, ordered = TRUE, levels = c( 1, 2, 3, 4))
icdata_cevd$ethnicity <- ifelse(icdata_cevd$ethnicity > 0, 1, 0)
icdata_cevdno$ethnicity <- ifelse(icdata_cevdno$ethnicity > 0, 1, 0)
icdata_cevd$sleep_filter <- ifelse(icdata_cevd$pa_sleep_convert_hr >= 7 & icdata_cevd$pa_sleep_convert_hr <= 9, 1, 0)
icdata_cevdno$sleep_filter <- ifelse(icdata_cevdno$pa_sleep_convert_hr >= 7 & icdata_cevdno$pa_sleep_convert_hr <=9, 1, 0)

In [243]:
model_dy <- polr(p20510 ~  card_sum_filter +hchol_yes +  diab_yes+ study_t2e+ white_yes + acc_age + male +sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = icdata_cevd, Hess = TRUE)
model_dn <- polr(p20510 ~ card_sum_filter +hchol_yes +  diab_yes+ study_t2e+ white_yes + acc_age + male+ sleep_filter +pa_sed_convert  + pa_light_convert + pa_mv_convert_log, data = icdata_cevdno, Hess = TRUE)

Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”
Warning message in polr(p20510 ~ card_sum_filter + hchol_yes + diab_yes + study_t2e + :
“design appears to be rank-deficient, so dropping some coefs”


In [244]:
df_model_dy <- summary(model_dy)
#get odds ratios
odds_ratios_dy <- exp(df_model_dy$coefficients[, "Value"])
#get standard errors
stde_dy <- df_model_dy$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dy <- exp(cbind(
  OR = df_model_dy$coefficients[, "Value"],
  lower = df_model_dy$coefficients[, "Value"] - 1.96 * stde_dy,
  upper = df_model_dy$coefficients[, "Value"] + 1.96 * stde_dy 
))
print(conf_int_dy)

                            OR        lower        upper
hchol_yes         1.341062e+00 6.151096e-01 2.923784e+00
diab_yes          4.153841e-01 9.488516e-02 1.818451e+00
study_t2e         1.276714e+00 7.747996e-01 2.103768e+00
white_yes         1.027310e+06 8.186117e+04 1.289214e+07
acc_age           9.710613e-01 9.250295e-01 1.019384e+00
male              1.157040e+00 5.784279e-01 2.314450e+00
sleep_filter      8.839301e-01 4.260109e-01 1.834067e+00
pa_sed_convert    1.002669e+00 9.980518e-01 1.007308e+00
pa_light_convert  1.001551e+00 9.969191e-01 1.006205e+00
pa_mv_convert_log 8.729114e-01 6.498660e-01 1.172510e+00
1|2               1.349623e+06 1.075454e+05 1.693685e+07
2|3               1.154464e+07 8.542253e+05 1.560229e+08
3|4               1.812344e+07 1.301690e+06 2.523329e+08


In [245]:

#make a table of coefficients
ctable_dy <- coef(summary(model_dy))
#get p values
p_dy <- pnorm(abs(ctable_dy[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dy <- cbind(ctable_dy, "p value" = p_dy))

,Value,Std. Error,t value,p value
hchol_yes,0.293461961,0.397661619,0.7379690,4.605333e-01
diab_yes,-0.878551546,0.753334931,-1.1662164,2.435270e-01
study_t2e,0.244289607,0.254816569,0.9586881,3.377159e-01
white_yes,13.842454176,1.290650083,10.7251798,7.753615e-27
acc_age,-0.029365643,0.024777566,-1.1851706,2.359500e-01
male,0.145865373,0.353727888,0.4123661,6.800711e-01
sleep_filter,-0.123377341,0.372404601,-0.3312992,7.404185e-01
pa_sed_convert,0.002665812,0.002355043,1.1319589,2.576517e-01
pa_light_convert,0.001549892,0.002365095,0.6553192,5.122622e-01
pa_mv_convert_log,-0.135921263,0.150544807,-0.9028625,3.665989e-01


In [246]:
df_model_dn <- summary(model_dn)
#get odds ratios
odds_ratios_dn <- exp(df_model_dn$coefficients[, "Value"])
#get standard errors
stde_dn <- df_model_dn$coefficients[, "Std. Error"]
#get 95% confidence intervals
conf_int_dn <- exp(cbind(
  OR = df_model_dn$coefficients[, "Value"],
  lower = df_model_dn$coefficients[, "Value"] - 1.96 * stde_dn,
  upper = df_model_dn$coefficients[, "Value"] + 1.96 * stde_dn 
))
print(conf_int_dn)

                          OR      lower     upper
hchol_yes         1.44938634 1.06633421 1.9700397
diab_yes          1.08030865 0.63885276 1.8268165
study_t2e         0.80928135 0.66877001 0.9793147
white_yes         1.23814908 0.60366927 2.5394918
acc_age           0.95973262 0.94540321 0.9742792
male              0.68419161 0.51996750 0.9002835
sleep_filter      0.90034726 0.68119283 1.1900084
pa_sed_convert    0.99989659 0.99870811 1.0010865
pa_light_convert  0.99949158 0.99802862 1.0009567
pa_mv_convert_log 0.87807646 0.78395583 0.9834971
1|2               0.09937998 0.08810998 0.1120915
2|3               0.74915513 0.56204519 0.9985557
3|4               2.18626251 1.36538363 3.5006599


In [247]:
#make a table of coefficients
ctable_dn <- coef(summary(model_dn))
#get p values
p_dn <- pnorm(abs(ctable_dn[, "t value"]), lower.tail = FALSE) * 2
#get table
(ctable_dn <- cbind(ctable_dn, "p value" = p_dn))

,Value,Std. Error,t value,p value
hchol_yes,0.3711402521,0.1565884991,2.3701629,1.778025e-02
diab_yes,0.0772467881,0.2680245236,0.2882079,7.731876e-01
study_t2e,-0.2116086415,0.0972991958,-2.1748242,2.964330e-02
white_yes,0.2136175867,0.3665032596,0.5828532,5.599921e-01
acc_age,-0.0411005588,0.0076751047,-5.3550486,8.553340e-08
male,-0.3795172722,0.1400365812,-2.7101295,6.725694e-03
sleep_filter,-0.1049747511,0.1423138320,-0.7376286,4.607402e-01
pa_sed_convert,-0.0001034188,0.0006067912,-0.1704356,8.646675e-01
pa_light_convert,-0.0005085509,0.0007473350,-0.6804858,4.961969e-01
pa_mv_convert_log,-0.1300216022,0.0578474450,-2.2476637,2.459764e-02
